In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

In [ ]:
mydata = pd.read_csv("stpsf-data/WFI/wim_zernikes_cycle9.csv", sep=",", header=0)
print(mydata.values)

In [ ]:
wavelength = mydata["wavelength"]
mask1 = np.where(wavelength == 0.48)
mask2 = np.where(wavelength == 0.62)

In [ ]:
mask1

In [ ]:
localAngleX = mydata["axis_local_angle_x"]
localAngleY = mydata["axis_local_angle_y"]
globalX = mydata["global_x"]
globalY = mydata["global_y"]
# sca = mydata['sca']

In [ ]:
localAngleXArray = np.array(localAngleX)[mask1]
localAngleYArray = np.array(localAngleY)[mask1]
globalXArray = np.array(globalX)[mask1]
globalYArray = np.array(globalY)[mask1]

In [ ]:
angleInput = np.dstack((localAngleXArray, localAngleYArray))
poly = PolynomialFeatures(degree=4)
angleInput_ = poly.fit_transform(angleInput[0])
modelX = linear_model.LinearRegression()
modelY = linear_model.LinearRegression()
modelX.fit(angleInput_, globalXArray)
modelY.fit(angleInput_, globalYArray)

In [ ]:
poly.powers_

In [ ]:
poly.powers_[:, 0]

In [ ]:
modelX.coef_

In [ ]:
modelY.coef_

In [ ]:
localAngleXArray = np.array(localAngleX)[mask2]
localAngleYArray = np.array(localAngleY)[mask2]
globalXArray = np.array(globalX)[mask2]
globalYArray = np.array(globalY)[mask2]

In [ ]:
angleInput = np.dstack((localAngleXArray, localAngleYArray))
poly = PolynomialFeatures(degree=4)
angleInput_ = poly.fit_transform(angleInput[0])
modelX = linear_model.LinearRegression()
modelY = linear_model.LinearRegression()
modelX.fit(angleInput_, globalXArray)
modelY.fit(angleInput_, globalYArray)

In [ ]:
modelX.coef_

In [ ]:
modelY.coef_

In [ ]:
def evalJ(jacobian, polyorder, x, y):
    evalArray = np.array([x, y])
    return np.sum(jacobian * np.prod(np.power(np.array(evalArray), polyorder), axis=3), axis=2)


def computeJacobian(wavelength, polyorder=4, tolerance=1e-4):
    # Read Input csv from STSCi Files
    mydata = pd.read_csv("stpsf-data/WFI/wim_zernikes_cycle9.csv", sep=",", header=0)

    # Define mask to desired wavelength
    mask1 = np.where(mydata["wavelength"] == wavelength)

    # Read columns from csv
    localAngleX = mydata["axis_local_angle_x"]
    localAngleY = mydata["axis_local_angle_y"]
    globalX = mydata["global_x"]
    globalY = mydata["global_y"]

    # Mask columns at desired wavelength
    localAngleXArray = np.array(localAngleX)[mask1]
    localAngleYArray = np.array(localAngleY)[mask1]
    globalXArray = np.array(globalX)[mask1]
    globalYArray = np.array(globalY)[mask1]

    # Sklearn multivariable polynomial fit
    angleInput = np.dstack((localAngleXArray, localAngleYArray))
    poly = PolynomialFeatures(degree=polyorder)
    angleInput_ = poly.fit_transform(angleInput[0])
    modelX = linear_model.LinearRegression()
    modelY = linear_model.LinearRegression()
    modelX.fit(angleInput_, globalXArray)
    modelY.fit(angleInput_, globalYArray)

    rx = modelX.score(angleInput_, globalXArray)
    ry = modelY.score(angleInput_, globalYArray)
    print("rx,ry =", rx, ry)
    if 1 - rx > tolerance or 1 - ry > tolerance:
        raise Exception("Not able to find a good model fit, please try again with higher polynomial order")

    coefX = modelX.coef_
    coefY = modelY.coef_
    xpowers = poly.powers_[:, 0]
    ypowers = poly.powers_[:, 1]
    newpowersx = np.clip(xpowers - 1, 0, None)
    newpowersy = np.clip(ypowers - 1, 0, None)

    newpolyorder = np.empty((2, 2, 15, 2), dtype=object)
    newpolyorder[0][0] = np.stack((newpowersx, ypowers), axis=1)
    newpolyorder[0][1] = np.stack((xpowers, newpowersy), axis=1)
    newpolyorder[1][0] = np.stack((newpowersx, ypowers), axis=1)
    newpolyorder[1][1] = np.stack((xpowers, newpowersy), axis=1)

    jacob = np.empty((2, 2, 15), dtype=object)
    jacob[0][0] = xpowers * coefX
    jacob[0][1] = ypowers * coefX
    jacob[1][0] = xpowers * coefY
    jacob[1][1] = ypowers * coefY

    return jacob, newpolyorder

In [ ]:
j1, poly1 = computeJacobian(0.869)

In [ ]:
j2, poly2 = computeJacobian(0.48)

In [ ]:
print(len(j1[0][0]))
print(j2)

In [ ]:
poly2.shape

In [ ]:
np.shape(np.array([2, 2]))

In [ ]:
np.power(np.array([2, 2]), poly2)

In [ ]:
np.sum(j1 * np.prod(np.power(np.array([2, 2]), poly2), axis=3), axis=2)

In [ ]:
jacob1 = evalJ(j1, poly1, -0.00522, -0.0981)

In [ ]:
det = -jacob1[0][0] * jacob1[1][1] + jacob1[0][1] * jacob1[1][0]

In [ ]:
np.sqrt(det) * 180 / np.pi